In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import numpy as np

# Load the IMDB dataset
(train_data, train_labels), (test_data, test_labels) = tf.keras.datasets.imdb.load_data(num_words=10000)



17464789/17464789 [==============================] - 2s 0us/step


In [2]:
def preprocess_data(sequences, maxlen, padding='post'):
    return tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=maxlen, padding=padding)

maxlen = 100

train_data = preprocess_data(train_data, maxlen)
test_data = preprocess_data(test_data, maxlen)


In [3]:
class TransformerModel(tf.keras.Model):
    def __init__(self, num_heads, num_layers, d_model, vocab_size, dff, dropout):
        super(TransformerModel, self).__init__()
        self.embedding = layers.Embedding(input_dim=vocab_size, output_dim=d_model)
        self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.dense1 = layers.Dense(units=d_model)
        self.norm1 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(dropout)
        self.ffn = layers.Dense(units=dff, activation='relu')
        self.dense2 = layers.Dense(units=d_model)
        self.norm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout2 = layers.Dropout(dropout)
        self.global_pool = layers.GlobalAveragePooling1D()
        self.fc_out = layers.Dense(units=1, activation='sigmoid')

    def call(self, x, training):
        x = self.embedding(x)
        attn_output = self.attention(x, x)
        x1 = self.dense1(attn_output)
        x = self.norm1(x + x1)
        x = self.dropout1(x, training=training)
        ffn_output = self.ffn(x)
        x2 = self.dense2(ffn_output)
        x = self.norm2(x + x2)
        x = self.dropout2(x, training=training)
        x = self.global_pool(x)
        return self.fc_out(x)

# Define the model architecture
num_heads = 2
num_layers = 2
d_model = 64
vocab_size = 10000
dff = 512
dropout = 0.1

model = TransformerModel(num_heads, num_layers, d_model, vocab_size, dff, dropout)



In [4]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
loss = tf.keras.losses.BinaryCrossentropy()

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Train the model
epochs = 10
history = model.fit(train_data, train_labels, epochs=epochs, validation_data=(test_data, test_labels), batch_size=64)




Epoch 1/10
391/391 [==============================] - 50s 105ms/step - loss: 0.4064 - accuracy: 0.8100 - val_loss: 0.3446 - val_accuracy: 0.8471
Epoch 2/10
391/391 [==============================] - 12s 32ms/step - loss: 0.2616 - accuracy: 0.8931 - val_loss: 0.3620 - val_accuracy: 0.8448
Epoch 3/10
391/391 [==============================] - 9s 24ms/step - loss: 0.2181 - accuracy: 0.9160 - val_loss: 0.4089 - val_accuracy: 0.8319
Epoch 4/10
391/391 [==============================] - 8s 21ms/step - loss: 0.1884 - accuracy: 0.9272 - val_loss: 0.4561 - val_accuracy: 0.8251
Epoch 5/10
391/391 [==============================] - 8s 20ms/step - loss: 0.1602 - accuracy: 0.9408 - val_loss: 0.5359 - val_accuracy: 0.8246
Epoch 6/10
391/391 [==============================] - 9s 22ms/step - loss: 0.1335 - accuracy: 0.9483 - val_loss: 0.6192 - val_accuracy: 0.8154
Epoch 7/10
391/391 [==============================] - 8s 20ms/step - loss: 0.1035 - accuracy: 0.9568 - val_loss: 0.6787 - val_accuracy: 0.8